## [ISIC 2024 - Skin Cancer Detection with 3D-TBP](https://www.kaggle.com/competitions/isic-2024-challenge/overview)
##### Lingfeng (Ling) Su, Manal Amarsaikhan, Yuxin (Katy) Chen, Keelan Gan

In [46]:
import tensorflow as tf
import pandas as pd
import os

In [47]:
def load_and_preprocess_image(image_path, label):
    width, height = 100, 100
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    
    image = tf.image.resize(image, [width, height])
    image = image / 255.0
    
    return image, label

In [48]:
def batch_dataset(batch_size, dataset):
    return dataset.batch(batch_size)

#### Process Image Dataset

In [ ]:
# Load metadata
metadata = pd.read_csv(r"\train-metadata.csv", low_memory=False)
image_dir = r"\train-image\image"

# Create a mapping from image filenames to labels
labels = metadata[['isic_id', 'target']].copy()
labels['isic_id'] = labels['isic_id'].apply(lambda x: x + '.jpg')
image_to_label = dict(zip(labels['isic_id'], labels['target']))

# Create a list of image file paths and their corresponding labels
image_paths = [os.path.join(image_dir, img_id) for img_id in labels['isic_id']]
image_labels = [image_to_label[img_id] for img_id in labels['isic_id']]

# Create a TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((image_paths, image_labels))

# Parallelize image loading and preprocessing
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and repeat the dataset
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.repeat()

# Batch the dataset
batch_size = 1000
num_batches = 10
dataset = batch_dataset(batch_size, dataset)
dataset = dataset.take(num_batches)

In [50]:
# Print the first batch
for images, labels in dataset.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)
    

Images shape: (1000, 100, 100, 3)
Labels shape: (1000,)
